<a href="https://colab.research.google.com/github/MatchLab-Imperial/deep-learning-course/blob/master/07_VAE_GAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Variational Autoencoders

<a href="https://ibb.co/GMMVqft"><img src="https://i.ibb.co/X55zqf3/vae.jpg" alt="vae" border="0"></a>

Image taken from [here](http://kvfrans.com/variational-autoencoders-explained/)

In the autoencoder tutorial we showed how to learn a meaningful representation of the data by using an autoencoder. In an autoencoder, the input image was transformed into a vector which encoded the information from the image in a lower dimensionality space. Then, we decoded that vector to get a reconstruction of the input image. However, the model was focused on encoding existing data for representation learning or similar purposes. To tackle the generation of new data, we will use a Variational Autoencoder (VAE) approach. The image shows an overview of the VAE method.

Parts of the code are taken from [here](https://tiao.io/post/tutorial-on-variational-autoencoders-with-a-concise-keras-implementation/), which contains a more in-depth explanation.


Before starting to define the different parts of the VAE, let's import the needed modules for this tutorial.

In [ ]:
import datetime
import sys

from IPython.display import HTML, display, clear_output
from PIL import Image as pil_image
import ipywidgets as widgets
from matplotlib import animation
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d
import numpy as np
from scipy import stats
import tensorflow as tf
from tensorflow.keras import applications, backend, callbacks, datasets, initializers, layers, losses, models, optimizers, preprocessing
from tqdm import tqdm


%matplotlib inline
np.random.seed(123)  # for reproducibility
tf.config.run_functions_eagerly(False)

Now we load MNIST, which will be our toy dataset for this example.

In [ ]:
original_dim = 784
(x_train, y_train), (x_test, y_test) = datasets.mnist.load_data()
x_train = x_train.reshape(-1, original_dim) / 255.
x_test = x_test.reshape(-1, original_dim) / 255.

Now let's do a quick recap of the Variational AutoEncoder (VAE) theory. As stated in the lecture, we want to find the $\hat{\theta}$ that approximates $P_\theta(x)$ by doing:

$$
\hat{\theta} = \text{argmax}_\theta \sum_{i=1}^n \mathbb{E}_{Q_\phi(z|x_i)}[\log(P_\theta(x_i|z))] - \text{KL}(Q_\phi(z|x_i) || P_\theta(z))
$$
Se will minimize the negative of that term in order to maximize it, where
we will train our model using a stochastic approach by sampling mini-batches from the dataset. First, we define the two losses we will use. The loss `nll` is the first term of the equation, whereas the `KLDivergenceLayer` is the second term of the loss. The `KLDivergenceLayer`will be used to compute an extra loss in the middle of the model via the `self.add_loss` function, but it does not change its inputs.

In [ ]:
def nll(y_true, y_pred):
    """Negative log likelihood (Bernoulli)."""
    return tf.reduce_sum(backend.binary_crossentropy(y_true, y_pred), axis=-1)

class KLDivergenceLayer(layers.Layer):
    """Identity transform layer that adds KL divergence to the final model loss."""

    def call(self, inputs):
        mu, log_var = inputs
        kl_batch = -0.5 * tf.reduce_sum(1 + log_var - tf.square(mu) - tf.exp(log_var), axis=-1)
        self.add_loss(tf.reduce_mean(kl_batch))
        return inputs

In the last block, we just defined the two losses we will use. Here we build the whole VAE model. First, we build the encoder. The goal of the encoder $\phi$ is to approximate $P_\theta(z|x_i)$ via $Q_\phi(z|x_i)$.  We assume that $P(z)$ is a Normal distribution with zero mean and unit variance. We also assume that $Q(z|x)={N}(\mu_x, \sigma_x)$ so the encoder tries to recover the parameters $\mu_x, \sigma_x$ for the different $x$ (which are the input images), i.e. the encoder outputs for each latent dimension a mean and standard deviation. The code for this encoder is the following:


In [ ]:
# Define latent dimension
latent_dim = 2
original_dim = 28 * 28

# Encoder
x = layers.Input(shape=(original_dim,))
h = layers.Reshape((28, 28, 1))(x)
h = layers.Conv2D(64, kernel_size=(5, 5), strides=(2, 2), padding='same')(h)
h = layers.LeakyReLU(0.2)(h)
h = layers.Conv2D(128, kernel_size=(5, 5), strides=(2, 2), padding='same')(h)
h = layers.LeakyReLU(0.2)(h)
h = layers.Flatten()(h)

# Compute μ and log(σ²)
z_mu = layers.Dense(latent_dim)(h)
z_log_var = layers.Dense(latent_dim)(h)

# Apply KL divergence layer
z_mu, z_log_var = KLDivergenceLayer()([z_mu, z_log_var])

As we mentioned, the encoder will output the parameters $\mu_x, \sigma_x$. Now we will sample from the normal distribution defined by those parameters to pass it to the decoder. However, we now face one of the problems of implementing a VAE: we want to optimize both decoder and encoder at the same time to i) encourage good reconstruction and ii) to make $z$ follow a normal distribution. What is the problem here? Using a standard sampling method, i.e by directly sampling using the mean and standard deviation output by the encoder, we cannot train it in an end-to-end manner as sampling is not a differentiable operation.

We need a trick to solve this issue. We can use one of the properties of the Normal probability distribution, which is that sampling $\mathcal{N}(\mu_\psi, \sigma_\psi)$ is the same as $\mu_\psi + \sigma_\psi\mathcal{N}(0, 1)$. Now, the sampling process is just a factor multiplying by the prediction $\sigma_\phi$ of the encoder, meaning we can propagate the gradients from the output back to the encoder. This is called the reparametrisation trick. We use this trick in the following block.

In [ ]:
##### Reparametrisation trick
# Log_var to sigma
z_sigma = layers.Lambda(lambda t: tf.exp(0.5 * t))(z_log_var)

# Define a separate noise input (which must be provided during training)
eps = layers.Input(shape=(latent_dim,), name="eps_input")

# Multiply sigma with the external noise
z_eps = layers.Multiply()([z_sigma, eps])

# Add mu to obtain the latent vector
z = layers.Add()([z_mu, z_eps])

Now we define the decoder, which will take the sampling from $\mathcal{N}(\mu_\psi, \sigma_\psi)$ as input and output an image.

In [ ]:
decoder = models.Sequential()
decoder.add(layers.InputLayer(shape=(latent_dim,)))
decoder.add(layers.Dense(128*7*7, kernel_initializer=initializers.RandomNormal(stddev=0.02)))
decoder.add(layers.LeakyReLU(0.2))
decoder.add(layers.Reshape((7, 7, 128)))
decoder.add(layers.UpSampling2D(size=(2, 2)))
decoder.add(layers.Conv2D(64, kernel_size=(5, 5), padding='same'))
decoder.add(layers.LeakyReLU(0.2))
decoder.add(layers.UpSampling2D(size=(2, 2)))
decoder.add(layers.Conv2D(1, kernel_size=(5, 5), padding='same', activation='sigmoid'))
decoder.add(layers.Flatten())
x_pred = decoder(z)

We have defined both the encoder and the decoder, and we are ready to train the model. We now build the model, which will have two inputs: the image `x`  for the encoder; and the sample `eps` (which refers to the sample from $\mathcal{N}(0, 1)$ we mentioned before) for the decoder, which will use $\mu_x, \sigma_x$ via the reparametrisation trick.

In [ ]:
vae = models.Model(inputs=[x, eps], outputs=x_pred)
vae.compile(optimizer='adam', loss=nll)

Now we train the model for some epochs to see if we can model our data. After the training process we will use this trained VAE to generate new data.

In [ ]:
epochs = 20
batch_size = 50

early_stopping = callbacks.EarlyStopping(
    monitor='val_loss',  # Monitor validation loss
    patience=3,          # Number of epochs with no improvement after which training will be stopped
    restore_best_weights=True  # Restore model weights from the epoch with the best value of the monitored quantity
)


# Generate noise inputs for training and validation (one per sample)
eps_train = np.random.normal(size=(x_train.shape[0], latent_dim))
eps_test = np.random.normal(size=(x_test.shape[0], latent_dim))

history = vae.fit(
    [x_train, eps_train], x_train,
    shuffle=True,
    epochs=20,
    batch_size=50,
    validation_data=([x_test, eps_test], x_test),
    callbacks=[early_stopping]
)

Assuming we input into the decoder $\mu_x$ obtained after encoding the image with the encoder, we can retrieve the MSE in the test set using the following piece of code.

In [ ]:
encoder = models.Model(x, z_mu)
z_test = encoder.predict(x_test, batch_size=batch_size)
images = decoder.predict(z_test)
print("The MSE is: {:.4f}".format(((images-x_test)**2).mean()))

We trained an encoder to model $Q(z|x)$ which outputs two parameters per latent dimension for each image $x$, which are  $\mu_x, \sigma_x$. We now plot the distribution of the $\mu_x$ when encoding the different images from the test set.

In [ ]:
encoder = models.Model(x, z_mu)

# display a 2D plot of the digit classes in the latent space
z_test = encoder.predict(x_test, batch_size=batch_size)
fig, ax = plt.subplots(figsize=(6,6))
cm = plt.get_cmap('gist_rainbow')
ax.set_prop_cycle(color=[cm(1.*i/(10)) for i in range(10)])
for l in range(10):
    # Only select indices for corresponding label
    ind = y_test == l
    ax.scatter(z_test[ind, 0], z_test[ind, 1], label=str(l),s=10)
ax.legend()
plt.title("Latent distribution")
plt.show()

The distribution of the encoded means shows how they are clustered by class too, as in the autoencoder case. However, in this case, the distribution also follows a kind of circular distribution around the centre due to the Kullback Leibler divergence term in the loss. We made the $\mu_x$ output by the encoder to be close to zero, and $\sigma_x$ to be close to 1.

Now let's start with the generation of data, which is the main reason we trained this model.

In [ ]:
# display a 2D manifold of the digits
n = 15  # figure with 15x15 digits
digit_size = 28

# linearly spaced coordinates on the unit square were transformed
# through the inverse CDF (ppf) of the Gaussian to produce values
# of the latent variables z, since the prior of the latent space
# is Gaussian

z1 = stats.norm.ppf(np.linspace(0.01, 0.99, n))
z2 = stats.norm.ppf(np.linspace(0.01, 0.99, n))
z_grid = np.dstack(np.meshgrid(z1, z2))

x_pred_grid = decoder.predict(z_grid.reshape(n*n, latent_dim)).reshape(n, n, digit_size, digit_size)

plt.figure(figsize=(10, 10))
plt.imshow(np.block(list(map(list, x_pred_grid))), cmap='gray')
plt.axis('off')
plt.show()

You can see in the image how there is a smooth transition between the different generated numbers.

Now let's generate a nice animation for the latent variable, where we show the point we used in $z$ to generate the data and the corresponding image generated.

In [ ]:
## We create a 2d array
# Number of points to use, increase it for smoother animation
# Using more points makes the function slower
n_points = 50
# theta from 0 to 2pi
theta = np.linspace(0, 2*np.pi, n_points)
# radius of the circle (change it depending on your reprentation space plot)
r = 1
# compute x and y (you can add an offset depending on your latent space)
offset_x = 0
offset_y = 0
x = r*np.cos(theta) + offset_x
y = r*np.sin(theta) + offset_y
latent = np.stack([x, y], -1)

## We now plot as before the 2d scatter with the images from the test set
## and the corresponding label
z_test = encoder.predict(x_test, batch_size=batch_size)
fig, ax = plt.subplots(1,2)
ax = fig.add_subplot(121)
cm = plt.get_cmap('gist_rainbow')
ax.set_prop_cycle(color=[cm(1.*i/(10)) for i in range(10)])
for l in range(10):
    # Only select indices for corresponding label
    ind = y_test == l
    ax.scatter(z_test[ind, 0], z_test[ind, 1], label=str(l),s=10)
## This is needed to print the circle point (black point in the animation)
scat = ax.scatter(latent[0,0], latent[0,1], s=200, c='k')

## Create second plot with the generated image
fig.add_subplot(122)
latent_im = decoder.predict(latent)
im = plt.imshow(latent_im[0].reshape(28, 28), animated=True, cmap='gray')
def updatefig(i):
    global latent_im
    global latent, scat
    scat.set_offsets(latent[i])
    im.set_array(latent_im[i].reshape(28, 28))
    return im,

anim = animation.FuncAnimation(fig, updatefig, interval=200, save_count=latent.shape[0])
HTML(anim.to_html5_video())

# Generative Adversarial Networks



![](http://2018.igem.org/wiki/images/4/48/T--Vilnius-Lithuania-OG--introduction1.png)
[Image source](https://2018.igem.org/Team:Vilnius-Lithuania-OG/Gan_Introduction)

Generative Adversarial Networks have been shown to improve the generation of data compared to approaches such as VAE.

As you learnt in the lecture, we have two networks playing what is called a min max game between them. The Generator, $G$, tries to generate data that looks similar to real data, whereas the discriminator $D$ tries to distinguish between real and fake data.

Code adapted from [here](https://www.kdnuggets.com/2016/07/mnist-generative-adversarial-model-keras.html).


We first define $G$ here, and in this case we will use a convolutional network. Notice that we use what is called LeakyReLU for the activation functions, which have been shown to work well when using GANs. $G$ will take a vector of noise (in this case of dimensionality 10) sampled from $\mathcal{N}(0, 1)$ and output a generated image. We use `tanh` as the last activation because the data will be normalized to be between $[-1, 1]$.

In [ ]:
np.random.seed(123)  # for reproducibility

original_dim = 784
(x_train, y_train), (x_test, y_test) = datasets.mnist.load_data()
x_train = x_train.reshape(-1, original_dim) / 255.
x_test = x_test.reshape(-1, original_dim) / 255.

# Build Generative model
randomDim = 5

generator = models.Sequential([
    layers.Input(shape=(randomDim,)),
    layers.Dense(128 * 7 * 7, kernel_initializer=initializers.RandomNormal(stddev=0.02)),
    layers.LeakyReLU(0.2),
    layers.Reshape((7, 7, 128)),
    layers.UpSampling2D(size=(2, 2)),
    layers.Conv2D(64, kernel_size=(5, 5), padding='same'),
    layers.LeakyReLU(0.2),
    layers.UpSampling2D(size=(2, 2)),
    layers.Conv2D(1, kernel_size=(5, 5), padding='same', activation='tanh')
])

optimizer_gen = optimizers.Adam(learning_rate=2e-4, beta_1=0.5)
generator.compile(loss='binary_crossentropy', optimizer=optimizer_gen)
generator.summary()

Here, we define the discriminator $D$. The discriminator is trained with the `categorical_crossentropy` loss, but you could also use a binary loss, as its job is to discriminate between real and fake data.

In [ ]:
discriminator = models.Sequential([
    layers.Input(shape=(28, 28, 1)),
    layers.Conv2D(64, kernel_size=(5, 5), strides=(2, 2), padding='same', kernel_initializer=initializers.RandomNormal(stddev=0.02)),
    layers.LeakyReLU(0.2),
    layers.Dropout(0.4),
    layers.Conv2D(128, kernel_size=(5, 5), strides=(2, 2), padding='same'),
    layers.LeakyReLU(0.2),
    layers.Dropout(0.4),
    layers.Flatten(),
    layers.Dense(1, activation='sigmoid')
])
optimizer_disc = optimizers.Adam(learning_rate=2e-4, beta_1=0.5)
discriminator.compile(loss='binary_crossentropy', optimizer=optimizer_disc)
discriminator.summary()

We defined the combined network, which combines the generator and discriminator in a network. The discriminator, however, will not be updated when using this combined network. We will explain later why, but basically we will use the `discriminator` object (not the combined network) whenever we need to update the discriminator.

In [ ]:
# Combined network
discriminator.trainable = False

gan_input = layers.Input(shape=(randomDim,))
gen_output = generator(gan_input)
gan_output = discriminator(gen_output)

gan = models.Model(gan_input, gan_output)
optimizer_gan = optimizers.Adam(learning_rate=2e-4, beta_1=0.5)
gan.compile(loss='binary_crossentropy', optimizer=optimizer_gan)
gan.summary()

Now, let's define some helper functions that will be used to plot the loss and some generated images during training.

In [ ]:
def plot_loss(losses):
    plt.figure()
    plt.plot(losses["d"], label='discriminative loss')
    plt.plot(losses["g"], label='generative loss')
    plt.legend()
    plt.savefig('./loss.png')
    plt.close()

def plot_gen(mnist=1, n_ex=16, dim=(4,4), figsize=(10,10)):
    noise = np.random.normal(0,1,size=[n_ex,randomDim])
    generated_images = generator.predict(noise, verbose=0)

    plt.figure(figsize=figsize)
    for i in range(generated_images.shape[0]):
        plt.subplot(dim[0],dim[1],i+1)
        if mnist:
          img = generated_images[i,:,:,0]
          plt.imshow(img, cmap='gray')
        else:
          img = generated_images[i,:,:,:]
          plt.imshow(img)
        plt.axis('off')
    plt.tight_layout()
    plt.savefig('./images.png')
    plt.close()

We will not use the regular fit method to train the model as in the usual approach. The reason for this is that the min max game that the GANs do, in practice is implemented in each of the batches by using two different training steps.
* **First step - Training the discriminator:** In this step only the discriminator is trained.  The generator will output some fake images using noise as input. Then, we will give the discriminator these generated images and some images sampled from the real dataset and train it to distinguish between the two of them.

* **Second step - Updating the generator**: We use the generator to output fake images again, and the discriminator will try to guess if these newly generated images are real or fake. However, the aim in this second step is not to update the discriminator, only the generator. This is why in the combined network we made the discriminator not to be trainable. The discriminator is used to pass information (in the form of gradients in this case) to update the generator. Hence, the generator will try to change its weights to make the discriminator think the data comes from the real distribution.

In [ ]:
progress_out = widgets.Output()
plot_out = widgets.Output()

def update_plots(iteration, mnist_flag=1):
    global saved_images, saved_iterations
    with plot_out:
        plot_out.clear_output(wait=True)
        # Call your function that saves the image to './images.png'
        plot_gen(mnist_flag)
        fig, ax = plt.subplots(1, 1, figsize=(5, 5))
        img = mpimg.imread('images.png')
        ax.imshow(img)
        ax.axis('off')
        ax.set_title(f'Iteration: {iteration}')
        plt.tight_layout()
        display(fig)
        plt.close(fig)
    saved_images.append(img)
    saved_iterations.append(iteration)

def train_epoch(gan, generator, discriminator, epoch_number, plt_frq=50, BATCH_SIZE=32, mnist_flag=1):
    global total_iter

    vector_ind = np.random.permutation(x_train.shape[0])
    nb_epoch = int(x_train.shape[0] / BATCH_SIZE)
    pbar = tqdm(total=nb_epoch, leave=True)

    for e in range(nb_epoch):
        total_iter += 1

        ind = vector_ind[e * BATCH_SIZE:(e + 1) * BATCH_SIZE]
        image_batch = x_train[ind, :, :, :]
        noise_gen = np.random.normal(0, 1, size=[BATCH_SIZE, randomDim])
        generated_images = generator.predict(noise_gen, verbose=0)

        # --- Train the discriminator ---
        discriminator.trainable = True
        X = np.concatenate((image_batch, generated_images))
        y = np.zeros([2 * BATCH_SIZE])
        y[:BATCH_SIZE] = 1  # real images
        y[BATCH_SIZE:] = 0  # fake images
        d_loss = discriminator.train_on_batch(X, y)
        losses["d"].append(d_loss)

        # --- Train the generator ---
        discriminator.trainable = False
        noise_tr = np.random.normal(0, 1, size=[BATCH_SIZE, randomDim])
        y2 = np.ones([BATCH_SIZE])
        g_loss = gan.train_on_batch(noise_tr, y2)
        losses["g"].append(g_loss)

        with progress_out:
            pbar.update(1)
            pbar.set_postfix({'epoch': epoch_number, 'd_loss': d_loss, 'g_loss': g_loss})

        if e % plt_frq == 0:
            update_plots(total_iter, mnist_flag)

    with progress_out:
        pbar.close()

Let's start the training process. We will use MNIST again. A standard practice for GANs is to normalize images to the range $[-1, 1]$, which we do. We train it for 10 epochs.

In [ ]:
# Create separate output areas
display(progress_out)
display(plot_out)

# set up loss storage vector
losses = {"d":[], "g":[]}
(x_train, y_train), (x_test, y_test) = datasets.mnist.load_data()
x_train = x_train.reshape(-1, 28, 28, 1) / 127.5 - 1
x_test = x_test.reshape(-1, 28, 28, 1) / 127.5 - 1
n_epoch = 10

total_iter = 0
saved_images = []
saved_iterations = []

for e in range(n_epoch):
  train_epoch(gan, generator, discriminator, e+1, plt_frq=200,BATCH_SIZE=64)

In [ ]:
plot_loss(losses)
fig, ax = plt.subplots(1, 1, figsize=(7, 7))
img = mpimg.imread('loss.png')
ax.imshow(img)
ax.axis('off')
plt.tight_layout()

In [ ]:
# --- Create an animation from the captured images ---
fig_anim, ax_anim = plt.subplots(figsize=(5, 5))
im_anim = ax_anim.imshow(saved_images[0])
ax_anim.axis('off')
title_text = ax_anim.set_title(f'Iteration: {saved_iterations[0]}')

def update_frame(i):
    im_anim.set_array(saved_images[i])
    title_text.set_text(f'Iteration: {saved_iterations[i]}')
    return im_anim, title_text

anim = animation.FuncAnimation(fig_anim, update_frame, frames=len(saved_images),
                               interval=500, blit=False, repeat=False)

# Display the animation as an HTML5 video
HTML(anim.to_html5_video())

The plots show how the training loss for both the generator and the discriminator does not change much. Usually we have a model which we optimize to reduce some metric/loss we pass it. However, in this case we have two models which are 'competing' against each other, so their losses are approximately stable. Now let's plot some results, as in the VAE case, we plot the results when sampling from 2 of the dimensions of the input noise $z$. However, as in this case we used more than 2 dimensions for $z$, the results vary due to choosing at random the 2 dimensions where to sample from.

In [ ]:
# display a 2D manifold of the digits
n = 15  # figure with 15x15 digits
digit_size = 28

# linearly spaced coordinates on the unit square were transformed
# through the inverse CDF (ppf) of the Gaussian to produce values
# of the latent variables z, since the prior of the latent space
# is Gaussian

z1 = stats.norm.ppf(np.linspace(0.01, 0.99, n))
z2 = stats.norm.ppf(np.linspace(0.01, 0.99, n))
z_grid = np.dstack(np.meshgrid(z1, z2))
z_grid = z_grid.astype(np.float32)

z_fill = np.random.normal(0,1,size=[randomDim-2])
z_fill = np.expand_dims(z_fill, 0)
z_fill = np.expand_dims(z_fill, 0)
z_fill = np.repeat(z_fill, n, 0)
z_fill = np.repeat(z_fill, n, 1)

z_grid = np.concatenate([z_grid, z_fill], -1)
np.random.shuffle(z_grid[:,:,-1])
x_pred_grid = generator.predict(z_grid.reshape(n*n, randomDim), verbose=0)
x_pred_grid = np.reshape(x_pred_grid, (n, n, digit_size, digit_size))

plt.figure(figsize=(10, 10))
plt.imshow(np.block(list(map(list, x_pred_grid))), cmap='gray')
plt.axis('off')
plt.show()

**Training instability**

GANs are quite difficult to train as there are several factors that can hurt their performance. Some of the most frequent issues are:

* [Mode collapsing](https://www.youtube.com/watch?v=ktxhiKhWoEE): The generator is not capable of creating diverse images, it only generates a limited set of images.
* Discriminator loss decreases quickly: in some cases the discriminator may be too powerful, so it quickly learns at the beginning which images are fake and which real, leading to small gradients passed to the generator.
* Hyper parameter sensitivity
* Non-convergence

There are some tricks that have been shown to improve convergence, some of them are shared in this [repo](https://github.com/soumith/ganhacks).



# Inception Score


The evaluation of a generative model quantitatively is a problem that is being research. The metric should evaluate the coverage, i.e. how diverse the generated data is, and sample quality, which is related to the visual quality of the sample. A widely used metric is the Inception Score, which is defined as:
$$
\text{IS}(G)= \exp(\mathbb{E}_{x\sim p_a}\ D_{KL}(p(y|x)||p(y)))
$$
Let's explain the term a little bit. We aim to have a high IS, which means that we want a high $D_{KL}$ between $p(y|x)$ and $p(y)$. Ideally, what we want is $p(y|x)$ to be "peaky" and $p(y)$ to be uniform. $p(y|x)$ is the predicted probability vector $y$ given a generated sample $x$ by a model, in this case an Inception model (hence, the name of the score). If the sample $x$ is of high quality, the model should classify it with confidence in one of the available classes, hence the "peaky" $p(y|x)$. The term $p(y)$ is related to the diversity of the data, and is the marginal probability computed as $\int_z p(y|x=G(z))dz$, hence we average the predicted probabilites of the Inception model for the given samples. If the data is diverse, we will see approximately obtain a flat $p(y)$.


Code to compute the inception score adapted from [here](http://bluewidz.blogspot.com/2017/12/inception-score.html).

In [ ]:
from tf_keras.models import load_model # use legacy keras, v2
!wget https://imperialcollegelondon.box.com/shared/static/5cc14wf0s4qwj65lec5852jlmxfy32m9.h5 -O inception_score_mnist.h5
inception_score_model = load_model('./inception_score_mnist.h5', safe_mode=False)

In [ ]:

def inception_score(x, resizer=None, batch_size=32, denorm_im=1):
    r = None
    n_batch = (x.shape[0]+batch_size-1) // batch_size
    for j in range(n_batch):
        x_batch = x[j*batch_size:(j+1)*batch_size, :, :, :]
        if denorm_im:
          x_batch = (x_batch + 1)/2
        r_batch = inception_score_model.predict(x_batch) # r has the probabilities for all classes
        r = r_batch if r is None else np.concatenate([r, r_batch], axis=0)
    p_y = np.mean(r, axis=0) # p(y)
    e = r*np.log(r/p_y) # p(y|x)log(P(y|x)/P(y))
    e = np.sum(e, axis=1) # KL(x) = Σ_y p(y|x)log(P(y|x)/P(y))
    e = np.mean(e, axis=0)
    return np.exp(e) # Inception score


def image_inception_score(generator, n_ex=10000, dim_random=10, input_noise=None, denorm_im=1):
    if input_noise is None:
      input_noise = np.random.normal(0,1,size=[n_ex,dim_random])
    x_pred = generator.predict(input_noise)
    if len(x_pred.shape)==2:
      x_pred = x_pred.reshape(n_ex, 28, 28, 1)
    return inception_score(x_pred, denorm_im=denorm_im)

Now let's check the Inception Score for the GAN model we have trained before.

In [ ]:
image_inception_score(generator, dim_random=randomDim, denorm_im=1)

Due to the way we defined the generator for the GAN, images are generated in the range $[-1, 1]$ as we use a `tanh` activation function. That is why we use the `denorm_im` variable to map them to the range $[0, 1]$.

The decoder of the VAE can also be evaluated using the same `image_inception_score` function. A VAE uses a prior $\mathcal{N}(0,1)$ as an extra constraint in the loss, hence if we sample from $\mathcal{N}(0,1)$ as done in `image_inception_score`, the decoder should be able to generate new samples. When using the function for the VAE, use `image_inception_score(..., denorm_im=0)` as the decoder in the VAE already uses the range $[0,1]$.

# Conditional Generative Adversarial Networks (cGANs)

A Conditional GAN (cGAN) is a variant of classical Generative Adversarial Networks, where the task is conditioned on some extra information instead of random noise. They were introduced in 2014, [link to the cGANs paper](https://arxiv.org/abs/1411.1784), and set a new state of the art in many image tasks.

In this setting, cGANs expect an input image instead of a random noise vector. A simple example of image translation is the drawing to real object transformation. To do so, imagine that we have two different paired domains, images of the real objects (domain A) and drawing of objects (domain B), and we want to go from one to the other. In classical GANs, we used these two domains for training the architectures, but here we can also condition the input to one of the images in the domain B, and use the relationship between the domains to guide the network to the desired result:

<a href="https://imgbb.com/"><img src="https://i.ibb.co/rF2w5Tt/Screenshot-from-2019-02-25-15-47-18.png" alt="Screenshot-from-2019-02-25-15-47-18" border="0"></a>

As in regular GANs, the discriminator learns to classify between fake (synthesized by the generator) and real {drawing, photo} tuples. Hence, the generator will try to fool the discriminator with better and better synthesised images. In contrast to GANS, as we feed the drawing directly to the generator, the information that the network can use to seek convergence is much wider.

## Image-to-image Translation with cGANs


[Image-to-image Translation with Conditional Adversarial Networks](https://arxiv.org/abs/1611.07004) is a paper presented at the Computer Vision and Pattern Recognition (CVPR) conference in 2017. In their paper, the authors introduced Pix2pix, a conditional GAN, to transform images from one domain to another. The authors in the paper investigated conditional adversarial networks as a general-purpose solution to image-to-image translation problems, see some examples from the paper of different Image-to-Image tasks:

>

![texto alternativo](https://i.ibb.co/Mf08787/examples.jpg)

In this tutorial, we will explain and implement Pix2pix model for colouring images step by step.

## Colouring Black and White Images
The tutorial aims to build the Pix2pix code for colouring images. We will use CIFAR10 dataset to perform the experiment, however, you could try to use any other dataset. For instance, check [this repo](https://github.com/kvfrans/deepcolor) which shows how to colour manga-style images.

The idea is quite intuitive, first of all, we will transform images into black and white (BW) and train a generator that will transform them back to RGB images. As in classical GANs, we use the discriminator to differentiate between *Fake* and *Real* images.

Thus, we have domain A (colour) and domain B (B&W).

## Dataset Generator

When training a cGAN, or any regular GAN architecture, we need to train independently the generator and discriminator networks. As seen, the training is a min-max game between the two nets.

Firstly, we aim to train the generator to be able to fool the discriminator, while in the next iteration, we will train the discriminator to be able to distinguish between the generated (fake) images and the real ones.

We are going to switch the training between networks in every step. Instead of training per a whole epoch as has been seen in the previous tutorials, we will generate batches and train first the discriminator and then switch to train the generator.

To accomplish that, we need to use the module provided by Keras *model.train_on_batch()* and code our training loop. In this case, we can not rely on Keras *model.fit()* module to handle the dataset as done previously. Moreover, we need to create a dataset generator to feed with batches our training loop.

Let's define our class *DataLoader()*:

In [ ]:
class DataLoader():
  def __init__(self, dataset_name, img_res=(32, 32)):
    self.dataset_name = dataset_name
    self.img_res = img_res
    self.load_dataset()

  def _start_epoch(self):

    random_idx = np.random.permutation(len(self.im_A_train))
    self.im_A_train_epoch = self.im_A_train[random_idx]

  def load_dataset(self):

    if self.dataset_name == 'CIFAR10':
      (im_A_train, y_train), (im_A_test, y_test) = datasets.cifar10.load_data()
    elif self.dataset_name == 'CIFAR100':
      (im_A_train, y_train), (im_A_test, y_test) = datasets.cifar100.load_data()
    else:
      raise Exception('Please, select a valid dataset')

    self.im_A_train = im_A_train.astype('float32')
    self.im_A_test = im_A_test.astype('float32')
    self.im_A_train /= 255.
    self.im_A_test /= 255.

  def get_dataset_shape(self, is_training=True):

    if is_training:
      return self.im_A_train.shape
    else:
      return self.im_A_test.shape

  def get_num_batches(self, batch_size):
    return int(self.im_A_train.shape[0] / batch_size)

  def set_image_transformations(self, convert_to_bw):

    self.convert_to_bw = convert_to_bw

  def load_batch(self, batch_size=1, is_training=True):

    if is_training:

      self._start_epoch()
      num_batches = int((self.im_A_train).shape[0] / batch_size)

    else:
      num_batches = int((self.im_A_test).shape[0] / batch_size)

    for idx_batch in range(num_batches):

      init = idx_batch * batch_size
      end = (idx_batch + 1) * batch_size

      if is_training:
        batch = self.im_A_train_epoch[init:end]
      else:
        batch = self.im_A_test[init:end]

      batch = np.reshape(batch, (batch_size, self.img_res[0], self.img_res[1], 3))

      # Convert to B&W
      batch_bw = self.convert_to_bw.predict(batch)

      yield [batch, batch_bw]

  def get_random_batch(self, batch_size=1, is_training=True):

    if is_training:
      random_idx = np.random.permutation(len(self.im_A_train))
      batch = self.im_A_train[random_idx[0:batch_size]]
    else:
      random_idx = np.random.permutation(len(self.im_A_test))
      batch = self.im_A_test[random_idx[0:batch_size]]

    batch = np.reshape(batch, (batch_size, self.img_res[0], self.img_res[1], 3))

    # Convert to B&W
    batch_bw = self.convert_to_bw.predict(batch)

    return [batch, batch_bw]

*DataLoader* class needs only the name of the dataset to load. Now, it can load CIFAR10 and CIFAR100. Let's see what some of *DataLoader*'s methods do:


*   *_start_epoch()* is a private method. It shuffles the dataset every time that *load_batch()* is called.
*   *load_dataset()* loads the dataset into *DataLoader* object.
*  *load_batch()* creates a python generator. In each iteration, it will return a batch from the shuffled dataset.

Let's now define the dataset class.

In [ ]:
# Load the data, shuffled and split between train and test sets
dataset_loader = DataLoader(dataset_name = 'CIFAR10')

training_shape = dataset_loader.get_dataset_shape()
test_shape = dataset_loader.get_dataset_shape(is_training=False)

print('Shape of Training Images: {}'.format(training_shape))
print('Shape of Test Images: {}'.format(test_shape))

## Defining the Domains

As explained above, we will go from B&W to RGB colour images. To get the B&W images, we need to define *transform_bw*, a method that will transform images from **domain A** (colour) to images in **domain B** (B&W). We can do that with lambda functions as seen in the *CNN Introduction* tutorial.

In [ ]:
# Convert to greyscale color space
inp = layers.Input(shape=(None, None, 3))
out = layers.Lambda(lambda image: tf.compat.v1.image.rgb_to_grayscale(image))(inp)

transform_bw = models.Model(inputs=inp, outputs=out)

# Set predefined transformation in DataLoader Class
dataset_loader.set_image_transformations(transform_bw)

We can visualise images from both domains before starting to code our cGAN. Rerun the following code to check different examples:

In [ ]:
# Load random batch from dataset
random_batch = dataset_loader.get_random_batch(batch_size=9)

# Repeat last dimension for visualization
tmp = np.repeat(random_batch[1], 3, axis=3)

N=3
start_val = 0
fig, axes = plt.subplots(N,N)
plt.suptitle('Domain A VS Domain B', fontsize=18)
for row in range(N):
  for col in range(N):
    idx = start_val+row+N*col

    im = np.concatenate((random_batch[0][idx], tmp[idx]), 1)
    axes[row,col].imshow(np.clip(im, 0, 1))
    axes[row,col].set_xticks([])
    axes[row,col].set_yticks([])

## Generator & Discriminator Models

For many image translation problems, there is a great deal of low-level information shared between the input and output. Thus, it is desirable to shuttle this information directly across the net. For example, in the case of image colourization, the input and output share the location of prominent edges. To give the generator a means to circumvent the bottleneck for information like this, we use an architecture with skip connections, following the general shape of the UNet introduced in previous tutorials.

The generator introduced in this tutorial is a simpler version of the one in the paper since we are using low-resolution images (32x32).

In Pix2pix, the authors found that mixing the GAN objective with a more traditional loss, such as L1 loss, improved the final performance. The discriminator’s job remains unchanged, but the generator is tasked to not only fool the discriminator but also to be near the ground-truth output in an L1 sense.

We can define our cGAN generator model.

In [ ]:
def build_generator(im_shape):

  img_B = layers.Input(shape=(im_shape[0], im_shape[1], 1))

  ## Encoder part
  conv1 = layers.Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(img_B)
  pool1 = layers.MaxPooling2D(pool_size=(2, 2))(conv1)
  conv2 = layers.Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool1)
  pool2 = layers.MaxPooling2D(pool_size=(2, 2))(conv2)
  conv3 = layers.Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool2)
  pool3 = layers.MaxPooling2D(pool_size=(2, 2))(conv3)
  conv4 = layers.Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool3)
  drop4 = layers.Dropout(rate=0.5)(conv4)
  pool4 = layers.MaxPooling2D(pool_size=(2, 2))(drop4)

  conv5 = layers.Conv2D(1024, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool4)
  drop5 = layers.Dropout(rate=0.5)(conv5)

  ## Now the decoder starts
  up6 = layers.Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(layers.UpSampling2D(size = (2,2))(drop5))
  merge6 = layers.concatenate([drop4,up6], axis = 3)
  conv6 = layers.Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge6)

  up7 = layers.Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(layers.UpSampling2D(size = (2,2))(conv6))
  merge7 = layers.concatenate([conv3,up7], axis = 3)
  conv7 = layers.Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge7)

  up8 = layers.Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(layers.UpSampling2D(size = (2,2))(conv7))
  merge8 = layers.concatenate([conv2,up8], axis = 3)
  conv8 = layers.Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge8)

  up9 = layers.Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(layers.UpSampling2D(size = (2,2))(conv8))
  merge9 = layers.concatenate([conv1,up9], axis = 3)
  conv9 = layers.Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge9)
  conv10 = layers.Conv2D(3, 3,  padding = 'same')(conv9)

  model = models.Model(inputs = img_B, outputs = conv10, name='generator')

  return model

Next, we follow the discriminator model definition from the paper:

In [ ]:
def build_discriminator(im_shape):

  img_A = layers.Input(shape=(im_shape[0], im_shape[1], 3))
  img_B = layers.Input(shape=(im_shape[0], im_shape[1], 1))

  combined_imgs = layers.concatenate([img_A, img_B], axis=-1)

  disc_layer = layers.Conv2D(64, kernel_size=(5, 5), padding='same', strides=(2, 2))(combined_imgs)
  disc_layer = layers.LeakyReLU(negative_slope=0.2)(disc_layer)

  disc_layer = layers.Conv2D(64, kernel_size=(5, 5), padding='same', strides=(2, 2))(disc_layer)
  disc_layer = layers.BatchNormalization(momentum=0.8)(disc_layer)
  disc_layer = layers.LeakyReLU(negative_slope=0.2)(disc_layer)

  disc_layer = layers.Conv2D(128, kernel_size=(5, 5), strides=(2, 2))(disc_layer)
  disc_layer = layers.BatchNormalization(momentum=0.8)(disc_layer)
  disc_layer = layers.LeakyReLU(negative_slope=0.2)(disc_layer)

  disc_layer = layers.Flatten()(disc_layer)
  disc_layer = layers.Dense(1024)(disc_layer)

  prob = layers.Dense(1, name="disc_dense")(disc_layer)

  discriminator = models.Model(inputs=[img_A, img_B], outputs=[prob], name='discriminator')

  return discriminator


## Training Loop

We arrive at the tricky part of the cGAN tutorial, where we need to define the training loop of our model. Do not focus much on this part of the code, this is only provided in case you want to further train the architectures.

First, we need to define the optimisers, inputs, outputs, and losses for each one of the networks. Let's go line by line:

In [ ]:
# Define optimizers for each network.
# Note that if the discriminator is able to differentiate between samples,
# the generator will be not able to learn. They must learn at the same time.
# Setting the learning rates or learning steps is always tricky.
optimizer_g = optimizers.Adam(learning_rate=2e-4, beta_1=0.5)
optimizer_d = optimizers.Adam(learning_rate=2e-5, beta_1=0.5)

# Input size
im_shape = (32, 32)

# Build and compile the discriminator
discriminator = build_discriminator(im_shape)
discriminator.compile(loss='mse', optimizer=optimizer_d, metrics=['accuracy'])

# Build the generator
generator = build_generator(im_shape)

# Input images and their conditioning images
img_A = layers.Input(shape=(im_shape[0], im_shape[1], 3))
img_B = layers.Input(shape=(im_shape[0], im_shape[1], 1))

# By conditioning on B generate a fake version of A.
# Remember, images in B are the B&W version of images in A
fake_A = generator(img_B)

# For the combined model we will only train the generator, thus, we freeze the
# discriminator model when optimizing G.
# We are backpropagating the discriminator's error
# into the generator in order to create fake images that are unrecognizable by D
discriminator.trainable = False

# Discriminators determines if a conditioned pair has been generated or is real
valid = discriminator([fake_A, img_B])

# Minimize discriminator error (only updating generator's weights) and
# the L1 loss between real and fake images.
combined = models.Model(inputs=[img_A, img_B], outputs=[valid, fake_A])
combined.compile(loss=['mse', 'mae'], loss_weights=[1, 100], optimizer=optimizer_g)

We have created the models and compiled them. Besides, before starting the training loop, we are going to define two auxiliary functions that will allow us to print images during training to have an idea of how well the generator is colouring the images:

In [ ]:
def showColoredIms(imB, fake_imA, real_imA):

  plt.subplot(131)
  plt.imshow(np.clip(imB[0], 0, 1)[:,:,0], cmap='gray')
  plt.title('Domain B', fontsize=20)
  plt.gca().set_xticks([])
  plt.gca().set_yticks([])
  plt.subplot(132)
  plt.imshow(np.clip(fake_imA[0], 0, 1))
  plt.title('Fake A', fontsize=20)
  plt.gca().set_xticks([])
  plt.gca().set_yticks([])
  plt.subplot(133)
  plt.imshow(np.clip(real_imA[0], 0, 1))
  plt.title('Real A', fontsize=20)
  plt.gca().set_xticks([])
  plt.gca().set_yticks([])
  plt.show()

def showColored_two_models_Ims(imB, fake_imA_MAE, fake_imA_cGAN, real_imA):

  plt.subplot(141)
  plt.imshow(np.clip(imB[0], 0, 1)[:,:,0], cmap='gray')
  plt.title('BW', fontsize=20)
  plt.gca().set_xticks([])
  plt.gca().set_yticks([])
  plt.subplot(142)
  plt.imshow(np.clip(fake_imA_MAE[0], 0, 1))
  plt.title('MAE', fontsize=20)
  plt.gca().set_xticks([])
  plt.gca().set_yticks([])
  plt.subplot(143)
  plt.imshow(np.clip(fake_imA_cGAN[0], 0, 1))
  plt.title('cGAN', fontsize=20)
  plt.gca().set_xticks([])
  plt.gca().set_yticks([])
  plt.subplot(144)
  plt.imshow(np.clip(real_imA[0], 0, 1))
  plt.title('Real', fontsize=20)
  plt.gca().set_xticks([])
  plt.gca().set_yticks([])
  plt.show()

In the next section, we provide trained weights so you could resume or skip the training. These weights were obtained by training the net for 20 epochs. Note that if you want to further train the networks, GAN training is characterized for being really long.

Let's see the training loop.

In [ ]:
num_epochs = 1
batch_size = 128
n_batches = dataset_loader.get_num_batches(batch_size)

# Adversarial loss ground truths.
# They are used as labels for the discriminator loss.
valid = np.ones((batch_size,))
fake = np.zeros((batch_size,))

for epoch in range(num_epochs):

  start_time = datetime.datetime.now()

  # Record average losses. Monitorize the loss function.
  g_avg_loss = []
  d_avg_loss = []
  d_avg_acc = []

  # load_batch() returns a batch generator
  # Before starting the epoch, it shuffles the dataset
  for batch_i, [imgs_A, imgs_B] in enumerate(dataset_loader.load_batch(batch_size)):

    # ---------------------
    #  Train Discriminator
    # ---------------------

    # Condition on B and generate a translated (fake) version of A.
    # It will try to color images from B to be as similiar as possible
    # to their correspoding pairs from A.
    fake_A = generator.predict(imgs_B)

    # Train the discriminator (original images = real / generated = Fake)
    d_loss_real = discriminator.train_on_batch([imgs_A, imgs_B], valid)
    d_loss_fake = discriminator.train_on_batch([fake_A, imgs_B], fake)
    d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)
    d_avg_loss.append(d_loss[0])
    d_avg_acc.append(d_loss[1])

    # -----------------
    #  Train Generator
    # -----------------

    # Train the generators
    g_loss = combined.train_on_batch([imgs_A, imgs_B], [valid, imgs_A])
    g_avg_loss.append(g_loss[0])

    elapsed_time = datetime.datetime.now() - start_time

    # Aproximation of epoch remaining time
    remaining_time = (elapsed_time/(batch_i+1)) * (n_batches-batch_i-1)

    # Plot examples
    if batch_i%50 == 0:
      showColoredIms(imgs_B, fake_A, imgs_A)

    # Plot the progress
    if batch_i%10 == 0:

      print ("[Epoch %d/%d] [Batch %d/%d] [D loss: %f, acc: %3d%%] [G loss: %f] elapsed_time: %s  remaining_time: %s" % (epoch, num_epochs,
              batch_i, n_batches, np.mean(d_avg_loss), 100*np.mean(d_avg_acc), np.mean(g_avg_loss), elapsed_time, remaining_time))

  # Saves optimizer and weights
  generator.save('generator.h5')
  discriminator.save('discriminator.h5')

If desired, you could download the model you just trained. Those weights can be used for resuming future trainings.

In [ ]:
from google.colab  import files

# Download the weights in your PC
files.download('generator.h5')
files.download('discriminator.h5')

## Colouring Test Images

We are ready to visualise how the network colours the test images. If you have not to train your model, you can optionally load the provided trained model.

In [ ]:
# Load weights from a previous session
# Upload weights to Colab by using the File tab
from tf_keras.models import load_model # use legacy keras under tf_keras

!wget -O weights.zip https://imperialcollegelondon.box.com/shared/static/w2m93zcadbycpspaq4jifug9bo8l88ad.zip
!unzip -q ./weights.zip
!rm ./weights.zip

generator = load_model('./weights/generator.h5')
discriminator = load_model('./weights/discriminator.h5')

Let's see the results, you can rerun the following code for visualizing more examples:

In [ ]:
# Load random batch from dataset

for i in range(3):
  [im_A_real, im_B_test] = dataset_loader.get_random_batch(batch_size=1, is_training=False)

  im_A_fake = generator.predict(im_B_test)

  showColoredIms(im_B_test, im_A_fake, im_A_real)

The colouring may not be perfect, but the network does a good job at guessing plausible colours. We have shown a simplified version of the Pix2pix method. Training GANs is a hard task, and there are many elements that could be added to improve results. For instance, [this repo](https://github.com/soumith/ganhacks), like many others, shows common tricks to improve GAN performance.

# Coursework



## Task 1: MNIST generation using VAE and GAN

**Report**
* Train the given VAE model in the tutorial also using the same hyper parameters (batch size, optimizer, number of epochs, etc...) but increasing the latent dimensionality to 10. Compute the MSE on the reconstructed `x_test` images and Inception Score (IS). Now train the same model without the KL divergence loss, and compute again the MSE and IS score. Report the results in a table and discuss them. For the IS you can use in this case `image_inception_score(decoder, dim_random=10, denorm_im=0)`.

* Train the GAN given in the tutorial with increased dimensionality of the initial random sample to 10 for 10 epochs and report its IS (use the same table as in the VAE case). Discuss the difference in the obtained IS for VAE and GAN and link it to the qualitative results. For the IS use in this case `image_inception_score(generator, dim_random=10, denorm_im=1)`.



## Task 2: Quantitative VS Qualitative Results

In this task, we will observe the difference between two trained models for colouring images. One is the model trained during the tutorial, which uses a cGAN approach to predict the RGB pixel-wise values of a B&W image. The other one is a simple UNet autoencoder trained with a Mean Absolute Error (MAE) loss, which is trained to predict directly the RBG image without any GAN based learning strategy. We refer to the first and second models as cGAN and MAE models, respectively. For this task, 20 epochs trained weights for the cGAN and MAE models are provided. If desired, the code to train the MAE model can be found below:

In [ ]:
# Input images and their conditioning images
img_A = layers.Input(shape=(im_shape[0], im_shape[1], 3))
img_B = layers.Input(shape=(im_shape[0], im_shape[1], 1))

# Build the generator
generator_mae = build_generator(im_shape)
fake_A = generator_mae(img_B)
generator_mae = models.Model(inputs=img_B, outputs=fake_A)
generator_mae.compile(optimizer = 'Adam', loss = 'mean_absolute_error', metrics = ['mae'])

num_epochs = 1
batch_size = 128
n_batches = dataset_loader.get_num_batches(batch_size)

for epoch in range(num_epochs):

  start_time = datetime.datetime.now()

  # Record average losses. Monitorize the loss function.
  g_avg_loss = []

  # load_batch() returns a batch generator
  # Before starting the epoch, it shuffles the dataset
  for batch_i, [imgs_A, imgs_B] in enumerate(dataset_loader.load_batch(batch_size)):

    # -----------------
    #  Train Generator (MAE)
    # -----------------
    g_loss = generator_mae.train_on_batch(imgs_B, imgs_A)
    g_avg_loss.append(g_loss[0])

    fake_A = generator_mae.predict(imgs_B)

    elapsed_time = datetime.datetime.now() - start_time

    # Aproximation of epoch remaining time
    remaining_time = (elapsed_time/(batch_i+1)) * (n_batches-batch_i-1)

    # Plot examples
    if batch_i%50 == 0:
      showColoredIms(imgs_B, fake_A, imgs_A)

    # Plot the progress
    if batch_i%10 == 0:

      print ("[Epoch %d/%d] [Batch %d/%d] [G loss: %f] elapsed_time: %s  remaining_time: %s" % (epoch, num_epochs,
              batch_i, n_batches, np.mean(g_avg_loss), elapsed_time, remaining_time))

  # Saves optimizer and weights
  generator_mae.save('generator_mae.h5')

Instead of training the models, we can directly load their pre-trained weights by running:

In [ ]:
!wget -O weights.zip https://imperialcollegelondon.box.com/shared/static/w2m93zcadbycpspaq4jifug9bo8l88ad.zip
!unzip -q ./weights.zip
!rm ./weights.zip

!wget -O weights_mae.zip https://imperialcollegelondon.box.com/shared/static/empxfp2v05xp7h8rwsccuiy35c3vddzd.zip
!unzip -q ./weights_mae.zip
!rm ./weights_mae.zip

generator_cGAN = models.load_model('./weights/generator.h5')
generator_mae = models.load_model('mae_generator.h5', compile=False)

generator_cGAN.compile(optimizer='Adam', loss='mean_absolute_error', metrics=['mae'])
generator_mae.compile(optimizer='Adam', loss='mean_absolute_error', metrics=['mae'])

We have loaded both models, and we are ready to compare them. In this task, you are asked to analyse the difference between the quantitative versus the qualitative results. To do so, we provided two pieces of code. The first one will compute the MAE metric for both models in the test dataset. As we know, this metric is widely used on image generation tasks, such as image upsampling, image reconstruction, image translation, and so on.

In [ ]:
g_mae_avg_mae = []
for batch_i, [imgs_A, imgs_B] in enumerate(dataset_loader.load_batch(128, is_training=False)):
    _, mae = generator_mae.evaluate(imgs_B, imgs_A, verbose=0)
    g_mae_avg_mae.append(mae)

print("MAE (Trained MAE): {:.4f}".format(np.mean(g_mae_avg_mae)))

g_cgan_avg_mae = []
for batch_i, [imgs_A, imgs_B] in enumerate(dataset_loader.load_batch(128, is_training=False)):
    _, mae = generator_cGAN.evaluate(imgs_B, imgs_A, verbose=0)
    g_cgan_avg_mae.append(mae)

print("MAE (Trained cGAN): {:.4f}".format(np.mean(g_cgan_avg_mae)))

The next piece of code will show coloured examples for both networks, so you can check them visually and discuss which model is better. First, we need to create an iterator object to go through the test dataset:

In [ ]:
iterator = iter(dataset_loader.load_batch(1, is_training=False))

Run multiple examples so that you have a clear idea of how both methods differ.

In [ ]:
# Load test example
[imgs_A, imgs_B] = next(iterator)

# Generate predictions for both models
fake_A_cGAN = generator_cGAN.predict(imgs_B)
fake_A_MAE = generator_mae.predict(imgs_B)

# Plot all images
showColored_two_models_Ims(imgs_B, fake_A_MAE, fake_A_cGAN, imgs_A)

We showed that both models obtain a similar MAE value. If we would only take into account the quantitative metric, as done in many scientific articles, we would say that the MAE model is better. However, in addition to the quantitative results, we need to analyse visually the results produced by the two networks to declare which is the best model.

**Report**


*   Run the previous code to analyse several coloured images for both models. Based on previous results and linked to GAN theory, discuss from the numerical and visual perspective if both models are similar, or whether there is a better one. You can provide in the report visual examples together with their MAE values to support your arguments. The figure of this task can be included in the Appendix. Discussion still needs to go into the main text.